# CSE 572: Lab 6

In this lab, you will practice implementing feed-forward network and parameter tuning with k-fold cross valivation.

To execute and make changes to this notebook, click File > Save a copy to save your own version in your Google Drive or Github. Read the step-by-step instructions below carefully. To execute the code, click on each cell below and press the SHIFT-ENTER keys simultaneously or by clicking the Play button.

When you finish executing all code/exercises, save your notebook then download a copy (.ipynb file). Submit the following **three** things:
1. a link to your Colab notebook,
2. the .ipynb file, and
3. a pdf of the executed notebook on Canvas.

To generate a pdf of the notebook, check the instructions on Canvas.

# **PUT YOUR GROUP INFO HERE**

| Group number | August Group XXX |            |
|--------------|------------------|------------|
| Member 1     | NAME             | ASURITE ID |
| Member 2     |                  |            |
| Member 3     |                  |            |
| Member 4     |                  |            |

In [ ]:
# Import libraries
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

# Set the random seed for reproducibility
seed = 0
tf.random.set_seed(seed)

## Feed-forward neural networks

### Load the dataset

For this example, we will use the [Cleveland Heart Disease dataset](https://archive.ics.uci.edu/ml/datasets/heart+Disease). Review the dataset documentation to learn more about the attributes and other aspects of the dataset. The dataset consists of a CSV file with 303 rows. Each row contains information about a patient. There are 14 attribute columns and one binary class column (`target`) that reports whether or not a patient had a heart disease. We will train a feed-forward neural network model to predict whether or not a given patient has a heart disease based on the attribute values.

In [ ]:
# Load the dataset
data = pd.read_csv("http://storage.googleapis.com/download.tensorflow.org/data/heart.csv")

In [ ]:
# Print sample rows
data.head()

In [ ]:
# Print the number of rows and columns
data.shape

Split the dataset into three randomly-sampled subsets: training (60%), validation (20%), and test (20%). Use the `seed` variable for the `random_state`.

In [ ]:
train, val, test = np.split(data.sample(frac=1, random_state=seed), [int(.6*len(data)), int(.8*len(data))])

Print the number of samples in each of the three subsets and the number of instances from each class. For example, for the training set you might print "The training set has __ instances (__ heart disease, __ no disease)".

In [ ]:
# YOUR CODE HERE


### Prepare the dataset

Before we can feed this dataset to our model for training and evaluation, we need to perform a few steps to get it ready:
1. Convert the dataframes to Dataset objects
2. Normalize the numerical feature values
3. Binarize the Categorical features by converting to one-hot encodings

In [ ]:
# Convert dataframes to Dataset objects
def dataframe_to_dataset(df, shuffle=True):
    df = df.copy()
    # Remove the target column and store in a separate array
    labels = df.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
      ds = ds.shuffle(buffer_size=len(df), reshuffle_each_iteration=False)

    return ds


train_ds_original = dataframe_to_dataset(train)
val_ds_original = dataframe_to_dataset(val)
test_ds_original = dataframe_to_dataset(test)

The Dataset object yields a tuple containing the input feature vector and target (class value): `(input, target)`. `input` is a dictionary of features and `target` is the value 0 or 1. The code below prints an example instance drawn from the training Dataset object.

In [ ]:
for x, y in train_ds_original.take(1):
    print("Input:", x)
    print("\nTarget:", y)

We can use the batch() function in keras to create batches from the full dataset for passing to the model. For the training dataset, we'll define a hyperparameter `batch_size` that we will set. For the validation and test sets, we will make the batch size equivalent to the size of the subset so all samples in that subset are evaluated each time the dataset is evaluated by the model.

In [ ]:
batch_size = 32
train_ds = train_ds_original.batch(batch_size)
val_ds = val_ds_original.batch(val.shape[0])
test_ds = test_ds_original.batch(test.shape[0])

There are seven categorical features in the dataset: `sex`, `cp`, `fbs`, `restecg`, `exang`, `ca`, and `thal`. You can read more about what these features mean in the [dataset documentation](https://archive.ics.uci.edu/ml/datasets/heart+Disease). All of them except `thal` have integer data type while `thal` has String data type. Below we define a function to encode these feature values as one-hot encodings using the [IntegerLookup()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/IntegerLookup) and [StringLookup()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup) layers. These layers create look-up tables for mapping a set of arbitrary integers or strings to a one-hot encoding. We use an `is_string` argument to indicate whether we should use the `StringLookup()` for `thal` or the `IntegerLookup()` for the remaining features.

In [ ]:
def encode_categorical_feature(feature, name, dataset, is_string):
    from tensorflow.keras.layers import IntegerLookup
    from tensorflow.keras.layers import StringLookup

    # Create lookup layer to turn categorical features into 1-hot integer encodings
    if is_string:
        lookup = StringLookup(output_mode="binary")
    else:
        lookup = IntegerLookup(output_mode="binary")

    # Prepare a Dataset that only yields the feature of interest
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Find the set of possible values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

The remaining features in the dataset (`age`, `trestbps`, `chol`, `thalach`, `oldpeak`, and `slope`) are all numerical measurements. You can read more about what these features mean in the [dataset documentation](https://archive.ics.uci.edu/ml/datasets/heart+Disease). We don't need to encode the numerical features, but we do want to scale them to the same range of values (e.g., using standardization or normalization). Below we define a function that uses the [Normalization()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization) layer to standardize the data (subtract the mean and divide by the standard deviation for each feature).

In [ ]:
def normalize(feature, name, dataset):
    from tensorflow.keras.layers import Normalization

    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields the feature of interest
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    norm_feature = normalizer(feature)
    return norm_feature

Now we can apply these functions to each of our features and return an encoded/preprocessed input layer. We first create tensor variables for each of the inputs, then apply the appropriate function, then concatenate all of these input layers for each feature together to form a single input feature layer.

In [ ]:
all_inputs = [
    keras.Input(shape=(1,), name="sex", dtype="int64"),
    keras.Input(shape=(1,), name="cp", dtype="int64"),
    keras.Input(shape=(1,), name="fbs", dtype="int64"),
    keras.Input(shape=(1,), name="restecg", dtype="int64"),
    keras.Input(shape=(1,), name="exang", dtype="int64"),
    keras.Input(shape=(1,), name="ca", dtype="int64"),
    keras.Input(shape=(1,), name="thal", dtype="string"),
    keras.Input(shape=(1,), name="age"),
    keras.Input(shape=(1,), name="trestbps"),
    keras.Input(shape=(1,), name="chol"),
    keras.Input(shape=(1,), name="thalach"),
    keras.Input(shape=(1,), name="oldpeak"),
    keras.Input(shape=(1,), name="slope"),
]

In [ ]:
feature_layer = layers.concatenate(
    [
        encode_categorical_feature(all_inputs[0], "sex", train_ds, False),
        encode_categorical_feature(all_inputs[1], "cp", train_ds, False),
        encode_categorical_feature(all_inputs[2], "fbs", train_ds, False),
        encode_categorical_feature(all_inputs[3], "restecg", train_ds, False),
        encode_categorical_feature(all_inputs[4], "exang", train_ds, False),
        encode_categorical_feature(all_inputs[5], "ca", train_ds, False),
        encode_categorical_feature(all_inputs[6], "thal", train_ds, True),
        normalize(all_inputs[7], "age", train_ds),
        normalize(all_inputs[8], "trestbps", train_ds),
        normalize(all_inputs[9], "chol", train_ds),
        normalize(all_inputs[10], "thalach", train_ds),
        normalize(all_inputs[11], "oldpeak", train_ds),
        normalize(all_inputs[12], "slope", train_ds)
    ]
)

### Build the model

Now that we've prepared our dataset, we can construct our neural network model. We construct the model by composing Layer objects starting with the input layer (which we've already defined as `feature_layer`) and ending with the output layer (which will be the final output of the model). In this example, we will only use [Dense()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) layers which are simple fully-connected feed-forward layers (i.e., each output from layer `i-1` is connected by a weight variable to every neuron in layer `i`). We will create a network with only one hidden layer (between the input and output layers).

The Dense() layer object allows us to specify the activation function to use using the `activation` argument. In class, we talked about several activation functions including sigmoid, sign, and tanh. Another commonly used activation function is the rectified linear unit, or "ReLU" function. Another commonly used activation function is the rectified linear unit, or "ReLU" function, which has the equation $a(z)=max(0,z)$. We will use `relu` as our activation function in this example for all layers except the final layer, which will use a `sigmoid` activation.

In [ ]:
# Create a variable for the number of units/neurons in the layer
h1_units = 32
# Create a Dense layer and append it to the input layer
h1_layer = layers.Dense(h1_units, activation='relu')(feature_layer)

# Create an output layer with one output representing the likelihood of
# heart disease and append it to the hidden layer
output_layer = layers.Dense(1, activation="sigmoid")(h1_layer)

# Build the model specifying the input and output layer
model = keras.Model(inputs=all_inputs, outputs=output_layer)

We can plot our completed model to visualize the input, hidden, and output layers.

In [ ]:
# `rankdir='LR'` is to make the graph horizontal
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

Next we compile the model by specifying the optimization technique and loss function to be used in model training. We can also specify the metric(s) that will be logged during training. We will use stochastic gradient descent (`sgd`) for the optimizer and binary cross entropy (log loss) as the loss function. We will log the accuracy metric during training. We can also specify the learning rate hyperparameter here.

In [ ]:
learning_rate = 0.01
model.compile(keras.optimizers.SGD(learning_rate=learning_rate), "binary_crossentropy", metrics=["accuracy"])

### Train the model

Now that we've constructed and compiled our model, we can train the model using our training dataset. This is done in keras using the `fit()` function, which also gives us an option to provide the validation dataset which will be used to evaluate validation accuracy after every epoch.

In [ ]:
model_result = model.fit(train_ds, epochs=100, validation_data=val_ds)

The `fit()` function returns a history attribute that gives the metrics recorded during training as a dictionary. We can print the dictionary keys to see which metrics were stored:

In [ ]:
model_result.history.keys()

Create a figure with two subplots. The first subplot should plot the training and validation loss (`loss` and `val_loss`) and the second subplot should plot the training and validation accuracy (`accuracy` and `val_accuracy`). Make sure you include the axis labels and a legend in each plot.

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10,5))

# YOUR CODE HERE

### Test the model

Finally, we evaluate our trained model on the held-out test set. First we predict the outputs for the test data.

In [ ]:
preds = model.predict(test_ds)

The model output from the final sigmoid layer is a value between 0 and 1 representing the likelihood that a given sample patient has heart disease. To get the predicted classes, we predict 1 if the output was >= 0.5 and 0 otherwise.

In [ ]:
pred_classes = [1 if p >= 0.5 else 0 for p in preds]

Compute and print the test accuracy.

In [ ]:
from sklearn.metrics import accuracy_score

# YOUR CODE HERE

**Question 1: There is a large difference between the training and validation accuracies and the test accuracy. What do you think could explain this difference?**

**Answer:**

YOUR ANSWER HERE

## Hyperparameter tuning with K-fold Cross Validation

### k-fold Cross validation

We will use 5-fold cross validation to train and evaluate our classifier. We will not do any model selection/hyperparameter tuning in this step, so we need to split our data into a training and test set.

To split the data into 3 folds we will shuffle the rows and then split them into $k$ equal groups.

In [ ]:
k = 3

# Note: np.split raises error if indices_or_sections is
# an integer and doesn't result in equal size splits
folds = np.split(data.sample(frac=1, random_state=seed), indices_or_sections=k)

Use a for loop to print the number of samples and number of samples from each class in each fold.

In [ ]:
folds[0][folds[0]['target']==0]

In [ ]:
# YOUR CODE HERE


### Train a Neural Network classifier

We will use the Neural Network (NN) implemented in previous section for our classification model. Use cross validation to train and evaluate the model. Set hyperparameters to `h1_units=32` and `h1_activation='relu'`, and tune the hypaerparameter `learning_rate` with candidate values `[0.01, 0.03]`.

Implement a for loop to iterate hyperparameters, and then implement another for loop to iterate through each fold, training a new NN model each iteration with one fold assigned to validation and the remaining folds assigned to training. Compute the validation accuracy for each iteration and append it to the `accuracies` list.

In [ ]:
from sklearn.metrics import accuracy_score

def create_model(h1_units, learning_rate, h1_activation):
  all_inputs = [
    keras.Input(shape=(1,), name="sex", dtype="int64"),
    keras.Input(shape=(1,), name="cp", dtype="int64"),
    keras.Input(shape=(1,), name="fbs", dtype="int64"),
    keras.Input(shape=(1,), name="restecg", dtype="int64"),
    keras.Input(shape=(1,), name="exang", dtype="int64"),
    keras.Input(shape=(1,), name="ca", dtype="int64"),
    keras.Input(shape=(1,), name="thal", dtype="string"),
    keras.Input(shape=(1,), name="age"),
    keras.Input(shape=(1,), name="trestbps"),
    keras.Input(shape=(1,), name="chol"),
    keras.Input(shape=(1,), name="thalach"),
    keras.Input(shape=(1,), name="oldpeak"),
    keras.Input(shape=(1,), name="slope"),
  ]
  feature_layer = layers.concatenate(
    [
        encode_categorical_feature(all_inputs[0], "sex", train_ds, False),
        encode_categorical_feature(all_inputs[1], "cp", train_ds, False),
        encode_categorical_feature(all_inputs[2], "fbs", train_ds, False),
        encode_categorical_feature(all_inputs[3], "restecg", train_ds, False),
        encode_categorical_feature(all_inputs[4], "exang", train_ds, False),
        encode_categorical_feature(all_inputs[5], "ca", train_ds, False),
        encode_categorical_feature(all_inputs[6], "thal", train_ds, True),
        normalize(all_inputs[7], "age", train_ds),
        normalize(all_inputs[8], "trestbps", train_ds),
        normalize(all_inputs[9], "chol", train_ds),
        normalize(all_inputs[10], "thalach", train_ds),
        normalize(all_inputs[11], "oldpeak", train_ds),
        normalize(all_inputs[12], "slope", train_ds)
    ]
  )
  h1_layer = layers.Dense(h1_units, activation=h1_activation)(feature_layer)
  output_layer = layers.Dense(1, activation="sigmoid")(h1_layer)
  model = keras.Model(inputs=all_inputs, outputs=output_layer)
  model.compile(keras.optimizers.SGD(learning_rate=learning_rate), "binary_crossentropy", metrics=["accuracy"])
  return model


h1_units=32
learning_rate=[0.01, 0.03]
h1_activation='relu'

accuracies = []

for lr in learning_rate:
  accuracies_per_param = []

  for i in range(len(folds)):
      # assign the folds to training and validation
      train = pd.concat(folds[0:i] + folds[i+1:])
      val = folds[i]
      train_ds_original = dataframe_to_dataset(train)
      val_ds_original = dataframe_to_dataset(val, shuffle=False)

      train_ds = train_ds_original.batch(32)
      val_ds = val_ds_original.batch(val.shape[0])

      model = create_model(h1_units, lr, h1_activation)

      model_result = model.fit(train_ds, epochs=10, validation_data=val_ds)

      # predict test set, the output is a probability, not an integer
      pred_val = model.predict(val_ds)

      # YOUR CODE HERE
      # convert the probability to integer - the class label

      # append accuracy to accuracies_per_param list


  accuracies.append(accuracies_per_param)

**Question 2: How many total combinations of the above hyperparameter choices are there?**

**Answer:**

YOUR ANSWER HERE



Print the mean and standard deviation of the accuracy from cross validation for all hyperparams(across all $k$ folds).

In [ ]:
print(accuracies)
acc_mean = np.mean(accuracies, axis=1)
acc_std = np.std(accuracies, axis=1)

print(pd.DataFrame.from_dict(
    { "Params": learning_rate,
    "Accucary Mean": acc_mean,
    "Accucary Std ": acc_std,
}) )


**Question 3: If you increased the number of folds, do you expect the standard deviation of the accuracy across $k$ folds to increase or decrease? Why?**

**Answer:**

YOUR ANSWER HERE
